In [3]:
import pandas as pd
import httpx
from httpx import ConnectTimeout, ReadTimeout, ReadError, ConnectError
import time
import asyncio
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import numpy as np


# Export data from google sheet document and conver to pandas DataFrme
def get_data_from_google_sheet(sheet_id: str, sheet_name: str):
    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    df = pd.read_csv(url)
    return df


# Create async client via httpx for multiple requests
async def get_async_client(url: str):
     async with httpx.AsyncClient() as client:
            try:
                return await client.get(url, timeout=10)
            except (ConnectTimeout, ConnectError, ReadTimeout, ReadError):
                print(url)
                return await client.get(url, timeout=None)
           
            
# Method for division data into data packages
def division_into_packages(data: list, len_of_package:int = 60):
    split_data = [data[x:x+len_of_package] for x in range(0, len(data), len_of_package)]
    return split_data


# Method to verify currupted image or not
def verify_image(image):
    try:
        width, height = Image.open(BytesIO(image.content)).size
        return f"{str(width)}x{str(height)}"
    except UnidentifiedImageError:
        return "Corrupted Image"
        

# Async method for parse and collect image size data
async def collect_image_size(package: list):
    images_size_data = []
    for image in package:
        respsonse = await asyncio.gather(*map(get_async_client,image))
        image_size = [verify_image(image) for image in respsonse]
        images_size_data.extend(image_size)
    return images_size_data


# Method to create excel file and write data
def create_and_write_data_to_excel(file_name: str, data: list, columns_to_convert: list):
    assert len(data) == len(columns_to_convert)
    df = pd.DataFrame(np.column_stack(data), columns=columns_to_convert)
        
    exel_file = df.to_excel(file_name+".xlsx")
    print("Created")
    return


if __name__ == "__main__":                                       
    tm1 = time.perf_counter()
    df = get_data_from_google_sheet("1QX2IhFyYmGDFMvovw2WFz3wAT4piAZ_8hi5Lzp7LjV0", "Parser_ImageSize")
    list_of_urls = list(df["image_url"].dropna())
    split_data = division_into_packages(list_of_urls)
    image_size_lst = await collect_image_size(split_data)
    create_and_write_data_to_excel("test3", data=[list_of_urls, image_size_lst], columns_to_convert=["image_url", "Size"])
    tm2 = time.perf_counter()
    print(f'Total time elapsed: {tm2-tm1:0.2f} seconds')

60
120
180
240
300
360
420
480
540
600
660
720
780
840
900
960
1020
1080
1140
1200
1260
1320
1380
1440
1500
1560
1620
1680
1740
1800
1860
1920
1980
2040
2100
2160
2220
2280
2340
2400
2460
2520
2580
2640
2700
2760
2820
2880
2940
3000
3060
3120
3180
3240
3300
3360
3420
3480
3540
3600
3660
3720
3780
3840
3900
3960
4020
4080
4140
4200
4260
4320
4380
https://data.sanitino.eu/PRODUCT-92420/6d1cec0443693f67133ad759?v=42be2d0e&size=feed-1080
https://data.sanitino.eu/PRODUCT-92406/5b1559012bf72f9050ac097c?v=f26bc747&size=feed-1080
4440
https://data.sanitino.eu/PRODUCT-87838/48f19eebdc40a88f67e0b0a4?size=feed-1080
4500
4560
4620
4680
4740
4800
4860
4920
4980
5040
5100
5160
5220
5280
5340
5400
5460
5520
5580
5640
5700
5760
5820
5880
5940
6000
6060
6120
6180
6240
6300
6360
6420
6480
6540
6600
6660
6720
6780
https://data.sanitino.eu/PRODUCT-59433/e90a7655d9075ae50de665ef?size=feed-1080
6840
6900
6960
7020
7080
7140
7200
7260
7320
7380
7440
7500
7560
7620
7680
7740
7800
7860
7920
7980
8040
https://d